# Naive Bayes for Text Classification

**Imports:**

In [2]:
import time
import os
import re
import glob
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from itertools import chain
from functools import reduce
from typing import Iterator, List, Tuple
from iterator_utils import chain_unique

## Dataset processing

For each line in each file of the 20 newsgroups dataset, we tokenized and generated relevant terms
using the `nltk` library. We striped e-mail strings, "From:" and "Subject:" headers, non alphabetic characters and
stop words. We used the porter stemmer.

In [2]:
enc = "utf-8"
_stemmer = nltk.stem.porter.PorterStemmer()
_stopwords = set(stopwords.words("english"))

def get_terms (text: str) -> Iterator[str]:
    """
    :param text: the text to tokenize and process
    :return: An iterator with terms
    """
    tokens = nltk.word_tokenize(text)
    filtered = [
        w.lower() for w in tokens if w not in _stopwords and str.isalnum(w)
    ]
    return map(_stemmer.stem, filtered)

def filter_text_words (text: str):
    """
    :param text: the string to process
    :return: the initial string without email patterns,
    headers and non alphabetic characters
    """
    no_headers = re.sub("From[ ]*:|Subject[ ]*:|Re[ ]*:", "", text).strip(" ")
    no_emails = re.sub("[A-za-z0-9_.-]+@[A-za-z0-9.]+", " ", no_headers)
    return re.sub("[^A-Za-z']", " ", no_emails).strip(" ")

def get_doc_words (filepath: str) -> str:
    """
    Opens a file from the 20N dataset and applies the
    get_terms and the filter_text_words functions
    :param filepath: the ṕath to the file to open
    :return: a string with all the terms separated by space
    """
    with open(filepath, "r", encoding=enc, errors="ignore") as file:
        return " ".join(
            chain.from_iterable(get_terms(filter_text_words(line)) for line in file)
        )

We built and store the bag of words for the dataset using the `pandas` dataframe. We simply take
all the documents from a class (represented by `class_data`), take the tokenized words from each document
and build a vector with the counts of each word in the document. Notice that the binary bag of words can
be constructed from this representation, simply by transforming the numbers greater than 1 to 1:

```
bow = bag_of_words(class_data)
binbow = bow.applymap(lambda cnt: 1 if cnt >= 1 else 0)
```

In [3]:
def bag_of_words (class_data: pd.DataFrame):
    """
    :param class_data: a dataframe with the information of the documents; namely the document_id (numeric),
    the tokenized words (the "text" column) and the class (numeric from 0 to 19)
    :return: a dataframe with the bag of words for all the documents precent in class_data
    """
    all_text = class_data["text"]
    words = [text.split() for text in all_text]
    vocab = list(chain_unique(words))
    counts = [np.array([ws.count(w) for w in vocab]) for ws in words]
    rows = np.vstack(counts)
    idx = range(0, rows.shape[0])
    return pd.DataFrame(
        index=idx,
        columns=vocab,
        data=rows
    )

We store all the bags of words for each class (folder) of the 20 newsgroups dataset, yielding 20 dataframes.
We first build the dataframe representation of each class as described previously, and we build the bag of words
using the `bag_of_words` function:

In [6]:
# filepaths
newsgroups_path = "../data/20news-18828"
vocab_size_path = f"{newsgroups_path}/20N_vocab_size"
newsgroups_all_files = glob.glob(f"{newsgroups_path}/*")
newsgroups_dir_paths = [
    dir for dir in newsgroups_all_files if os.path.isdir(dir)
]
# Classes names and numeric id's
classes = [dir.split("/")[-1] for dir in newsgroups_dir_paths]
class_ids = [i for i, _ in enumerate(classes)]
# columns for the dataframes representing a class of newsgroup
dataframe_cols = ["doc", "text", "class"]

def save_data ():
    for i, dir_path in enumerate(newsgroups_dir_paths):
        print(f"Writing data from {dir_path}")
        t0 = time.time()
        filepaths = glob.glob(f"{dir_path}/*")
        doc_names = [f.split("/")[-1] for f in filepaths]
        docs_text = [get_doc_words(f) for f in filepaths]
        class_names = [classes[i]]*len(doc_names)
        rows_data = [np.array(data) for data in zip(doc_names, docs_text, class_names)]
        rows = np.vstack(rows_data)
        idx = range(0, rows.shape[0])
        class_data = pd.DataFrame(index=idx, columns=dataframe_cols, data=rows)
        bow = bag_of_words(class_data)
        binbow = bow.applymap(lambda cnt: 1 if cnt >= 1 else 0)

        dir_name = dir_path.split("/")[-1]
        data_path = f"{dir_path}/{dir_name}.csv"
        bow_path = f"{dir_path}/{dir_name}_bow.csv"
        binbow_path = f"{dir_path}/{dir_name}_binbow.csv"
        class_data.to_csv(data_path, index=False)
        bow.to_csv(bow_path, index=False)
        binbow.to_csv(binbow_path, index=False)
        t1 = time.time()
        print(class_data)
        print(bow)
        print(binbow)
        print(f"Finished writing data at {dir_path} in: {t1 - t0}s")

## Model building

We write the characteristic functions for a Naive Bayes Classifier.

$cnt(w, c) = $ count of the word $w$ inside the class $c$

$P(w, c) = \frac{cnt(w, c) + 1}{count(c) + |V|}$

$P(c) = |D_c| / |D|$ where $D$ is the set of all documents and $D_c$ is the set of documents that belong to class
$c$

$L(c | d) = P(c) \prod_{w \in d} P(w|c)$ the likelihood of the class $c$ given the document $d$. In this case,
since the multiplication returns values that are two small, for this implementation we take the $log_{10}$ of the
likelihood:

$log_{10} L(c|d) = log_{10} P(c) + \sum_{w \in d} log_{10} P(w|c)$

In [5]:
def prob (
        word: str,
        V: int,
        bow: pd.DataFrame
):
    """
    Calculates the probability of the words given a class represented
    by the bag of words bow
    :param word: a string
    :param V: the size of the vocabulary
    :param bow: the bag of words from a determined class
    :return: the probability of the word given the class
    """
    count_wc = bow.get(word, np.zeros(1)).sum()
    count_c = len(bow.columns)
    return (count_wc + 1) / (count_c + V)

def likelihood (
        words: List[str],
        V: int,
        prior_c: float,
        bow: pd.DataFrame):
    """
    Calculates the likelihood: the probability of the class represented by a bag of words
    (bow) given a document represented by a list of words
    :param words: the document
    :param V: the size of the vocabulary
    :param prior_c: the probabilities for each class P(c)
    :param bow: the bag of words of the class
    :return: the likelihood
    """
    p = np.array([prob(w, V, bow) for w in words])
    return np.log10(prior_c) + np.sum(np.log10(p))

def classify (
        words: List[str],
        V: int,
        prior: np.ndarray,
        bows: List[pd.DataFrame]
):
    """
    Given a document represented by a list of words, and all the classes represented
    by their respective bag of words, returns the most likely class to which the document belongs.
    Return the argmax of the bag of words that maximizes the likelihood
    :param words: the document
    :param V: the size of the vocabulary
    :param prior: the probabilities for each class P(c)
    :param bows: the bag of words of the class
    :return: the most likely class
    """
    ls = np.array([
        likelihood(words, V, prior[c], bows[c]) for c in class_ids
    ])
    return np.argmax(ls)

Note that we need to compute the size of the vocabulary, which is the number of unique words within all the
20 bags of words (columns of the dataframe). We have a vocabulary of 80086 words and 18828 documents in total.
We also need to partition the dataset in a training, testing and development set. The following functions help to partition
the dataframes. 60% is delegated to the training set, 30% to the training set and 10% to the development set

In [6]:
vocab_size_file = open(vocab_size_path)
V = int(vocab_size_file.readline())
vocab_size_file.close()
docs_data_filepaths = [
    f"{dir_path}/{dir_name}.csv" for dir_path, dir_name in zip(newsgroups_dir_paths, classes)
]
bow_filepaths = [
    f"{dir_path}/{dir_name}_bow.csv" for dir_path, dir_name in zip(newsgroups_dir_paths, classes)
]
binbow_filepaths = [
f"{dir_path}/{dir_name}_binbow.csv" for dir_path, dir_name in zip(newsgroups_dir_paths, classes)
]
# Note that each dataframe is shuffled initially
docs_data = [
    pd.read_csv(data_path).sample(frac=1, random_state=1)
    for data_path in docs_data_filepaths
]
bows = [
    pd.read_csv(bow_path)
    for bow_path in bow_filepaths
]
N = sum([
    d.shape[0] for d in docs_data
])
prior = np.array([
    d.shape[0] / N for d in docs_data
])
print(N)

def partition_docs (docs: np.ndarray):
    """
    :param docs: the id's for the diferent documents in a dataframe
    :return: a partition of documents to index a training (60%), testing (30%) and development set (10%)
    """
    n = len(docs)
    train_size = int(round(0.6*n))
    test_size = int(round(0.3*n))
    train_docs = docs[:train_size]
    test_docs = docs[train_size:train_size + test_size]
    dev_docs = docs[train_size + test_size:]
    return train_docs, test_docs, dev_docs

def partition_data () -> Iterator[Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]]:
    """
    For each existing class, retreives the dataframes corresponding to the class data and
    the bag of words. uses the partition_docs function to get the row id's for each data set,
    and returns the corresponding dataframes
    :return:
    """
    for c in class_ids:
        data = docs_data[c]
        bow = bows[c]
        docs = data.index.to_numpy()
        train_docs, test_docs, dev_docs = partition_docs(docs)
        train = bow.iloc[train_docs]
        test = data.text[test_docs].to_frame()
        dev = data.text[dev_docs].to_frame()
        yield train, test, dev

18828


Finally the following function classifies the test set in accordance to the training set, constructing
the confusion matrix, where the rows are labeled as the true class, and the columns are labeled with the
predicted class

In [7]:
train_set, test_set, dev_set = tuple(map(
    list,
    zip(*partition_data())
))


def exec_classification (test_set: List[pd.DataFrame]):
    for c, test in enumerate(test_set):
        pred = test.assign(
            expected_c=c,
            predicted_c=test.text.apply(
                lambda txt: classify(txt.split(), V, prior, train_set)
            )
        )
        tp = pred[pred.expected_c == pred.predicted_c].count().expected_c
        count_pred = pred.groupby(["predicted_c"]).count()
        confusion_row = count_pred.reindex(class_ids, fill_value=0)
        print(f"confusion: {confusion_row}")
        print(f"Predicted correctly: {tp}")
        yield confusion_row.to_numpy().T[0]


confusion_matrix_rows = np.vstack(
    list(exec_classification(test_set))
)
true_labels = [f"{c}_true" for c in classes]
pred_labels = [f"{c}_pred" for c in classes]
confusion_matrix = pd.DataFrame(
    index=true_labels,
    columns=pred_labels,
    data=confusion_matrix_rows
)
print(confusion_matrix)
confusion_matrix_path = f"{newsgroups_path}/confmatrix_nb.csv"
confusion_matrix.to_csv(confusion_matrix_path)

confusion:              text  expected_c
predicted_c                  
0             267         267
1               0           0
2               0           0
3               0           0
4               0           0
5               0           0
6               4           4
7               4           4
8               0           0
9              12          12
10              4           4
11              0           0
12              1           1
13              5           5
14              0           0
15              0           0
16              0           0
17              0           0
18              0           0
19              0           0
Predicted correctly: 267
confusion:              text  expected_c
predicted_c                  
0               8           8
1             116         116
2               4           4
3               5           5
4               0           0
5               7           7
6              28          28
7              14      

## Metrics
With the confusion matrix we can calculate the precision, recall and f1 scores

**Macroaveraging:**

In [8]:
confusion_matrix_path = f"{newsgroups_path}/confmatrix_nb.csv"
confusion_matrix = pd.read_csv(confusion_matrix_path, index_col=0)

# Precision
tps = np.diag(confusion_matrix)
predicted = confusion_matrix.apply(np.sum, axis=0)
precision = tps / predicted
precision

sci.crypt_pred                   0.834375
misc.forsale_pred                0.950820
sci.med_pred                     0.917219
rec.sport.hockey_pred            0.960000
alt.atheism_pred                 0.912000
comp.sys.mac.hardware_pred       0.949153
comp.os.ms-windows.misc_pred     0.543379
talk.politics.mideast_pred       0.673913
soc.religion.christian_pred      0.521661
talk.politics.misc_pred          0.629508
talk.politics.guns_pred          0.754438
rec.motorcycles_pred             0.983806
comp.windows.x_pred              0.828671
comp.graphics_pred               0.685185
rec.sport.baseball_pred          0.996063
comp.sys.ibm.pc.hardware_pred    0.696552
sci.electronics_pred             0.816794
sci.space_pred                   0.891892
rec.autos_pred                   0.914179
talk.religion.misc_pred          0.961538
dtype: float64

In [11]:
# Recall
true_count = confusion_matrix.apply(np.sum, axis=1)
recall = tps / true_count
recall

sci.crypt_true                   0.898990
misc.forsale_true                0.397260
sci.med_true                     0.932660
rec.sport.hockey_true            0.960000
alt.atheism_true                 0.475000
comp.sys.mac.hardware_true       0.583333
comp.os.ms-windows.misc_true     0.804054
talk.politics.mideast_true       0.989362
soc.religion.christian_true      0.966555
talk.politics.misc_true          0.827586
talk.politics.guns_true          0.934066
rec.motorcycles_true             0.815436
comp.windows.x_true              0.806122
comp.graphics_true               0.760274
rec.sport.baseball_true          0.848993
comp.sys.ibm.pc.hardware_true    0.684746
sci.electronics_true             0.727891
sci.space_true                   0.891892
rec.autos_true                   0.824916
talk.religion.misc_true          0.132979
dtype: float64

In [19]:
# F1
P = precision.to_numpy()
R = recall.to_numpy()
f1_data = (2 * P * R) / (P + R)
f1 = pd.Series(index=precision.index, data=f1_data)
f1

sci.crypt_pred                   0.865478
misc.forsale_pred                0.560386
sci.med_pred                     0.924875
rec.sport.hockey_pred            0.960000
alt.atheism_pred                 0.624658
comp.sys.mac.hardware_pred       0.722581
comp.os.ms-windows.misc_pred     0.648501
talk.politics.mideast_pred       0.801724
soc.religion.christian_pred      0.677608
talk.politics.misc_pred          0.715084
talk.politics.guns_pred          0.834697
rec.motorcycles_pred             0.891743
comp.windows.x_pred              0.817241
comp.graphics_pred               0.720779
rec.sport.baseball_pred          0.916667
comp.sys.ibm.pc.hardware_pred    0.690598
sci.electronics_pred             0.769784
sci.space_pred                   0.891892
rec.autos_pred                   0.867257
talk.religion.misc_pred          0.233645
dtype: float64

In [21]:
# Macro-averaged scores:

print(f"Macro-average precision: {np.mean(precision)}")
print(f"Macro-average recall: {np.mean(recall)}")
print(f"Macro-average f1: {np.mean(f1)}")

Macro-average precision: 0.8210572381000043
Macro-average recall: 0.7631057915608275
Macro-average f1: 0.7567599379966041


**Microaveraging**

In [22]:
micro_precision = np.sum(tps) / np.sum(predicted)
micro_recall = np.sum(tps) / np.sum(true_count)
micro_f1 = (2 * micro_precision * micro_recall) / (micro_precision + micro_recall)

print(f"Micro-average precision: {np.mean(micro_precision)}")
print(f"Micro-average recall: {np.mean(micro_recall)}")
print(f"Micro-average f1: {np.mean(micro_f1)}")

Micro-average precision: 0.7769121813031161
Micro-average recall: 0.7769121813031161
Micro-average f1: 0.7769121813031161
